In [1]:
#import required libraries
import pandas as pd

In [2]:
#get the sentiment dataset
df_sentiment = pd.read_csv('imdb_labelled.txt',sep='\t',names=['comment','label'])

In [3]:
#view first 10 observations. 
# 1 indicates positive sentiment and 0 indicate negative sentiment
df_sentiment.tail(10)

,comment,label
740,Lange had become a great actress.,1
741,It looked like a wonderful story.,1
742,I never walked out of a movie faster.,0
743,I just got bored watching Jessice Lange take h...,0
744,"Unfortunately, any virtue in this film's produ...",0
745,"In a word, it is embarrassing.",0
746,Exceptionally bad!,0
747,All in all its an insult to one's intelligence...,0
748,Not Good,0
749,Not Bad,1


In [4]:
# view more information about the setiment data using describe method
df_sentiment.describe()

,label
count,750.000000
mean,0.516000
std,0.500077
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [5]:
#view more info on data
df_sentiment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750 entries, 0 to 749
Data columns (total 2 columns):
comment    750 non-null object
label      750 non-null int64
dtypes: int64(1), object(1)
memory usage: 11.8+ KB


In [6]:
# view data using group by and describe method
df_sentiment.groupby('label').describe()

comment                                
        count unique                 top freq
label                                        
0         363    362  Not recommended.      2
1         387    385             10/10      2

In [7]:
# Verify length of the messages and also add it also as a new column (feature)
df_sentiment['length'] =df_sentiment['comment'].apply(len)

In [8]:
# view first 5 messages with length
df_sentiment.head()

,comment,label,length
0,"A very, very, very slow-moving, aimless movie ...",0,87
1,Not sure who was more lost - the flat characte...,0,99
2,Attempting artiness with black & white and cle...,0,188
3,Very little music or anything to speak of.,0,44
4,The best scene in the movie was when Gerardo i...,1,108


In [9]:
#view first 
df_sentiment[df_sentiment['length']>50]['comment'].iloc[0]

'A very, very, very slow-moving, aimless movie about a distressed, drifting young man.  '

In [10]:
# start text processing with vectorizer 
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

In [11]:
# define a function to get rid of stopwords present in the messages
def message_text_process(mess):
    # Check characters to see if there are punctuations
    no_punctuation = [char for char in mess if char not in string.punctuation]
    # now form the sentence.
    no_punctuation = ''.join(no_punctuation)
    # Now eliminate any stopwords
    return [word for word in no_punctuation.split() if word.lower() not in stopwords.words('english')]

In [12]:
# bag of words by applying the function and fit the data (comment) into it
import string
from nltk.corpus import stopwords
bag_of_words = CountVectorizer(analyzer=message_text_process).fit(df_sentiment['comment'])

In [13]:
# apply transform method for the bag of words
comment_bagofwords = bag_of_words.transform(df_sentiment['comment'])

In [14]:
# apply tfidf transformer and fit the bag of words into it (transformed version)
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(comment_bagofwords)

In [15]:
# print shape of the tfidf 
comment_tfidf = tfidf_transformer.transform(comment_bagofwords)
print(comment_tfidf.shape)

(750, 3260)


In [16]:
#choose naive Bayes model to detect the spam and fit the tfidf data into it
from sklearn.naive_bayes import MultinomialNB
sentiment_detection_model = MultinomialNB().fit(comment_tfidf,df_sentiment['label'])

In [19]:
# check model for the predicted  and expected value say for comment# 1 and comment#5
comment = input('Please give us yours valuable comments... ')
#comment = df_sentiment['comment'][4]
bag_of_words_for_comment = bag_of_words.transform([comment])
tfidf = tfidf_transformer.transform(bag_of_words_for_comment)

predict_result = sentiment_detection_model.predict(tfidf)[0]



if(predict_result==1):
    print('You liked the movie')
else:
    print('I guess you don\'t like this movie')

#print ('predicted sentiment label ', sentiment_detection_model.predict(tfidf)[0])
#print 'expected sentiment label', df_sentiment.label[4]

Please give us yours valuable comments... I find Megan Fox so hot
You liked the movie
